<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_9_3__Streamlit_automatyzacja_spotkan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adaptacja: https://www.theunwindai.com/p/build-multi-agent-ai-meeting-preparation-assistant

# Setup

In [ ]:
!pip install -qq  crewai streamlit #==1.40.2

Instalacja dwóch bibliotek Pythona za pomocą narzędzia pip:

`crewai` - biblioteka do tworzenia systemów agentowych i automatyzacji zadań
`streamlit` - framework do tworzenia aplikacji webowych z interfejsem graficznym

Flaga `-qq` oznacza "bardzo cichy" (quiet) tryb instalacji - wyświetlane są tylko krytyczne błędy, pomijane są standardowe komunikaty i ostrzeżenia.

Komentarz `#==1.40.2` jest zakomentowaną wersją Streamlit - prawdopodobnie autor kodu wcześniej używał konkretnej wersji 1.40.2, ale teraz zdecydował się na instalację najnowszej dostępnej wersji.

In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴
up to date, audited 23 packages in 629ms
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠴

Ta komenda instaluje narzędzie `localtunnel` za pomocą menedżera pakietów npm (Node Package Manager).

`localtunnel` to narzędzie, które tworzy publiczny URL dla lokalnie uruchomionego serwera. Pozwala to na udostępnienie aplikacji działającej na lokalnym komputerze przez internet - inni użytkownicy mogą uzyskać do niej dostęp przez wygenerowany adres URL.

Jest to szczególnie przydatne podczas:
- testowania webhooków
- demonstrowania aplikacji klientom/współpracownikom
- debugowania aplikacji w różnych środowiskach
- udostępniania lokalnej aplikacji bez konieczności wdrażania jej na serwer produkcyjny

# App

In [ ]:
%%writefile muh_app.py

import streamlit as st
from crewai import Agent, Task, Crew, LLM
from crewai.process import Process
from crewai_tools import SerperDevTool
import os


# inicjalizacja aplikacji
st.set_page_config(page_title="AI Meeting Agent 📝", layout="wide")
st.title("Agentura przed spotkaniem 📝")


# API
st.sidebar.header("API")
openai_key = st.sidebar.text_input("OpenAI API Key", type="password")
serper_key = st.sidebar.text_input("Serper API Key", type="password")

os.environ["OPENAI_API_KEY"] = openai_key
os.environ["SERPER_API_KEY"] = serper_key

# konfiguracja narzedzi
muh_llm = LLM(model= "gpt-4o-mini" , temperature= 0.7, api_key=openai_key)
search_tool = SerperDevTool()

# parametry
company_name = st.text_input("Wprowadź nazwę firmy:")
meeting_objective = st.text_input("Wprowadź cel spotkania:")
attendees = st.text_area("Wprowadź uczestników i ich role (po jednym w linii):")
meeting_duration = st.number_input("Wprowadź czas trwania spotkania (w minutach):", min_value=15, max_value=180, value=60, step=15)
focus_areas = st.text_input("Wprowadź konkretne obszary zainteresowania lub obawy:")

# agenci
context_analyzer = Agent(
    role='Specjalista ds. Kontekstu ',
    goal='Analizuj i podsumowuj kluczowe informacje kontekstowe do spotkania',
    backstory='Jesteś ekspertem w szybkim rozumieniu złożonych kontekstów biznesowych i identyfikowaniu kluczowych informacji.',
    verbose=True,
    allow_delegation=False,
    llm = muh_llm,
    tools=[search_tool]
)

industry_insights_generator = Agent(
    role='Ekspert Branżowy',
    goal='Dostarcz dogłębną analizę branży i zidentyfikuj kluczowe trendy',
    backstory='Jesteś doświadczonym analitykiem branżowym ze zdolnością do dostrzegania pojawiających się trendów i możliwości.',
    verbose=True,
    allow_delegation=False,
    llm = muh_llm,
    tools=[search_tool]
)

strategy_formulator = Agent(
    role='Strateg Spotkania',
    goal='Opracuj dostosowaną strategię spotkania i szczegółową agendę',
    backstory='Jesteś mistrzem planowania spotkań, znanym z tworzenia wysoce efektywnych strategii i agend.',
    verbose=True,
    allow_delegation=False,
    llm = muh_llm,
)

executive_briefing_creator = Agent(
    role='Specjalista ds. Komunikacji',
    goal='Zsyntetyzuj informacje w zwięzłe i skuteczne briefingi',
    backstory='Jesteś ekspertem komunikacji, biegłym w przekładaniu złożonych informacji na jasne, praktyczne wskazówki.',
    verbose=True,
    allow_delegation=False,
    llm = muh_llm,
)

# zadania
context_analysis_task = Task(
    description=f"""
    Przeanalizuj kontekst spotkania z firmą {company_name}, biorąc pod uwagę:
    1. Cel spotkania: {meeting_objective}
    2. Uczestników: {attendees}
    3. Czas trwania spotkania: {meeting_duration} minut
    4. Konkretne obszary zainteresowania lub obawy: {focus_areas}

    Dokładnie zbadaj firmę {company_name}, w tym:
    1. Ostatnie wiadomości i komunikaty prasowe
    2. Kluczowe produkty lub usługi
    3. Głównych konkurentów

    Przedstaw kompleksowe podsumowanie swoich ustaleń, podkreślając najistotniejsze informacje dla kontekstu spotkania.
    Sformatuj wynik używając markdown z odpowiednimi nagłówkami i podtytułami.
    """,
    agent=context_analyzer,
    expected_output="Szczegółowa analiza kontekstu spotkania i tła firmy, zawierająca ostatnie wydarzenia, wyniki finansowe i znaczenie dla celu spotkania, sformatowana w markdown z nagłówkami i podtytułami."
)

industry_analysis_task = Task(
      description=f"""
      Na podstawie analizy kontekstu dla firmy {company_name} i celu spotkania: {meeting_objective}, przedstaw dogłębną analizę branży:
      1. Zidentyfikuj kluczowe trendy i wydarzenia w branży
      2. Przeanalizuj sytuację konkurencyjną
      3. Podkreśl potencjalne szanse i zagrożenia
      4. Przedstaw spostrzeżenia dotyczące pozycjonowania rynkowego

      Upewnij się, że analiza jest odpowiednia do celu spotkania i ról uczestników.
      Sformatuj wynik używając markdown z odpowiednimi nagłówkami i podtytułami.
      """,
      agent=industry_insights_generator,
      expected_output="Kompleksowy raport analizy branży, zawierający trendy, sytuację konkurencyjną, szanse, zagrożenia i istotne spostrzeżenia dla celu spotkania, sformatowany w markdown z nagłówkami i podtytułami."
  )

strategy_development_task = Task(
      description=f"""
      Wykorzystując analizę kontekstu i spostrzeżenia branżowe, opracuj dostosowaną strategię spotkania i szczegółową agendę dla {meeting_duration}-minutowego spotkania z firmą {company_name}. Zawrzyj:
      1. Agendę z określonymi ramami czasowymi i jasnymi celami dla każdej sekcji
      2. Kluczowe punkty do omówienia dla każdego elementu agendy
      3. Proponowanych mówców lub prowadzących dla każdej sekcji
      4. Potencjalne tematy do dyskusji i pytania napędzające rozmowę
      5. Strategie dotyczące konkretnych obszarów zainteresowania i obaw: {focus_areas}

      Upewnij się, że strategia i agenda są zgodne z celem spotkania: {meeting_objective}
      Sformatuj wynik używając markdown z odpowiednimi nagłówkami i podtytułami.
      """,
      agent=strategy_formulator,
      expected_output="Szczegółowa strategia spotkania i agenda z ramami czasowymi, zawierająca cele, kluczowe punkty do omówienia i strategie dotyczące konkretnych obszarów zainteresowania, sformatowana w markdown z nagłówkami i podtytułami."
  )

executive_brief_task = Task(
      description=f"""
      Dokonaj syntezy wszystkich zebranych informacji w kompleksowy, lecz zwięzły brief dla kierownictwa na spotkanie z firmą {company_name}. Utwórz następujące komponenty:

      1. Szczegółowe jednostronicowe podsumowanie dla kierownictwa zawierające:
          - Jasne określenie celu spotkania
          - Lista kluczowych uczestników i ich ról
          - Najważniejsze informacje o tle firmy {company_name} i istotny kontekst branżowy
          - Najważniejsze 3-5 celów strategicznych spotkania, zgodnych z głównym celem
          - Krótki przegląd struktury spotkania i kluczowych tematów do omówienia

      2. Szczegółowa lista kluczowych punktów do omówienia, każdy poparty:
          - Istotnymi danymi lub statystykami
          - Konkretnymi przykładami lub studiami przypadków
          - Powiązaniem z obecną sytuacją lub wyzwaniami firmy

      3. Przewidywanie i przygotowanie się na potencjalne pytania:
          - Lista prawdopodobnych pytań od uczestników na podstawie ich ról i celu spotkania
          - Przygotowanie przemyślanych, opartych na danych odpowiedzi na każde pytanie
          - Dołączenie wszelkich dodatkowych informacji lub kontekstu, które mogą być potrzebne

      4. Rekomendacje strategiczne i następne kroki:
          - Przedstawienie 3-5 możliwych do wdrożenia rekomendacji na podstawie analizy
          - Nakreślenie jasnych następnych kroków do wdrożenia lub działań następczych
          - Zaproponowanie harmonogramów lub terminów dla kluczowych działań
          - Identyfikacja potencjalnych wyzwań lub przeszkód i propozycje strategii ich łagodzenia

      Upewnij się, że brief jest kompleksowy, ale zwięzły, możliwy do wdrożenia i dokładnie zgodny z celem spotkania: {meeting_objective}. Dokument powinien być ustrukturyzowany dla łatwej nawigacji i szybkiego odniesienia podczas spotkania.
      Sformatuj wynik używając markdown z odpowiednimi nagłówkami i podtytułami.
      """,
      agent=executive_briefing_creator,
      expected_output="Kompleksowy brief dla kierownictwa zawierający podsumowanie, kluczowe punkty do omówienia, przygotowanie pytań i odpowiedzi oraz \
      rekomendacje strategiczne, sformatowany w markdown z głównymi nagłówkami (H1), nagłówkami sekcji (H2) i nagłówkami podsekcji (H3) tam, gdzie to właściwe.\
       Użyj punktów, list numerowanych i wyróżnień (pogrubienie/kursywa) dla kluczowych informacji."
  )

# sklad
meeting_prep_crew = Crew(
      agents=[context_analyzer, industry_insights_generator, strategy_formulator, executive_briefing_creator],
      tasks=[context_analysis_task, industry_analysis_task, strategy_development_task, executive_brief_task],
      verbose=True,
      process=Process.sequential
  )

# odpal po nacisnieciu guzika ;-)
if st.button("Przygotuj Spotkanie"):
    with st.spinner("Agenci AI przygotowują twoje spotkanie..."):
        result = meeting_prep_crew.kickoff()
    st.markdown(result)

st.sidebar.markdown("""
## Jak korzystać z aplikacji:
1. Wprowadź swoje klucze API w pasku bocznym.
2. Podaj wymagane informacje o spotkaniu.
3. Kliknij 'Przygotuj Spotkanie', aby wygenerować kompleksowy pakiet przygotowawczy do spotkania.

Agenci AI będą współpracować, aby:
- Przeanalizować kontekst spotkania i tło firmy
- Dostarczyć spostrzeżenia i trendy branżowe
- Opracować dostosowaną strategię spotkania i agendę
- Stworzyć brief dla kierownictwa z kluczowymi punktami do omówienia

Ten proces może potrwać kilka minut. Prosimy o cierpliwość!
""")


Overwriting muh_app.py


Ten kod tworzy aplikację Streamlit do automatycznego przygotowywania spotkań biznesowych za pomocą agentów AI.

W pierwszej części kodu zaimportowane są niezbędne biblioteki i skonfigurowana jest podstawowa struktura aplikacji z tytułem "Agentura przed spotkaniem".

W pasku bocznym aplikacji znajdują się pola do wprowadzenia kluczy API dla OpenAI i Serper. Te klucze są potrzebne do działania agentów AI i narzędzia wyszukiwania.

Główna część interfejsu zawiera pola do wprowadzenia:
- nazwy firmy
- celu spotkania
- listy uczestników i ich ról
- czasu trwania spotkania (15-180 minut)
- konkretnych obszarów zainteresowania

Kod definiuje czterech specjalistycznych agentów AI:
1. Specjalista ds. Kontekstu - analizuje informacje kontekstowe
2. Ekspert Branżowy - dostarcza analizę branży i trendów
3. Strateg Spotkania - tworzy strategię i agendę
4. Specjalista ds. Komunikacji - przygotowuje briefingi

Dla każdego agenta zdefiniowane są zadania z dokładnymi instrukcjami dotyczącymi oczekiwanych wyników. Zadania te obejmują:
- analizę kontekstu firmy i spotkania
- analizę branży i konkurencji
- opracowanie strategii i agendy
- stworzenie briefu dla kierownictwa

Agenci są zorganizowani w zespół (Crew), który wykonuje zadania sekwencyjnie.

Po kliknięciu przycisku "Przygotuj Spotkanie" uruchamiany jest proces przygotowania, a wyniki są wyświetlane w formacie markdown.

W pasku bocznym znajduje się też instrukcja obsługi aplikacji z opisem kroków i informacją o czasie oczekiwania na wyniki.

# Biegnij Forrest

In [ ]:
!streamlit run muh_app.py --server.address=localhost  &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.201.217.238
⠙your url is: https://solid-pumas-scream.loca.lt


Ta sekwencja komend uruchamia kilka procesów:

1. `streamlit run muh_app.py --server.address=localhost`
- Uruchamia aplikację Streamlit
- Parametr `--server.address=localhost` określa, że serwer ma być dostępny tylko lokalnie
- `&>/content/logs.txt` przekierowuje wszystkie komunikaty (standardowe wyjście i błędy) do pliku logs.txt
- `&` na końcu uruchamia proces w tle

2. `npx localtunnel --port 8501`
- Tworzy tunel przez localtunnel do portu 8501 (domyślny port Streamlit)
- Pozwala na dostęp do lokalnej aplikacji z internetu
- Wygeneruje publiczny URL dostępowy

3. `curl ipv4.icanhazip.com`
- Pobiera publiczny adres IP komputera
- Może być przydatne do weryfikacji połączenia lub konfiguracji

Znak `&` pomiędzy komendami sprawia, że wszystkie procesy uruchamiają się równolegle.